In [13]:
import os
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor

In [6]:
df = pd.read_pickle('../data/clean_train_data.pkl')

In [7]:
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,total_pixels,screen_size
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,2631,17,3,7,1,1,0,2,1799140,17.262677
2,563,1,0.5,1,2,1,41,0.9,145,5,...,2603,11,2,9,1,1,0,2,2167308,11.180340
3,615,1,2.5,0,0,0,10,0.8,131,6,...,2769,16,8,11,1,0,0,2,2171776,17.888544
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1411,8,2,15,1,1,0,1,1464096,8.246211
5,1859,0,0.5,1,3,0,22,0.7,164,1,...,1067,17,1,10,1,0,0,1,1660616,17.029386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,668,13,4,19,1,1,0,0,2309580,13.601471
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,2032,11,10,16,1,1,1,2,1797975,14.866069
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,3057,9,1,5,1,1,0,3,1416576,9.055385
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,869,18,10,19,1,1,1,0,225120,20.591260


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1935 entries, 1 to 1999
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1935 non-null   int64  
 1   blue           1935 non-null   int64  
 2   clock_speed    1935 non-null   float64
 3   dual_sim       1935 non-null   int64  
 4   fc             1935 non-null   int64  
 5   four_g         1935 non-null   int64  
 6   int_memory     1935 non-null   int64  
 7   m_dep          1935 non-null   float64
 8   mobile_wt      1935 non-null   int64  
 9   n_cores        1935 non-null   int64  
 10  pc             1935 non-null   int64  
 11  px_height      1935 non-null   int64  
 12  px_width       1935 non-null   int64  
 13  ram            1935 non-null   int64  
 14  sc_h           1935 non-null   int64  
 15  sc_w           1935 non-null   int64  
 16  talk_time      1935 non-null   int64  
 17  three_g        1935 non-null   int64  
 18  touch_screen 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.25, random_state=2)